# Bayesian Analysis on COVID-19 Antibody Test Results

A simple workbook to determine the probability of an antibody test result matching the truth

In [1]:
import pandas as pd

This function will calculate the general population's expected distribution of COVID-19 given the prevalence.
It also will calculate the expected number of true positive, true negative, false positive and false negative results expected for a given test.

In [5]:
columns = ['COVID-19', 'Not COVID-19', 'Total']
index = ['Test +', 'Test -', 'Total']

def case_generator(p, sens, spec, n=10000, columns=columns, index=index):
    '''Takes prevalence (p - decimal) and the antibody test's sensitivity 
    (true positive detection rate; sens - decimal) and specificity (true 
    negative detection rate; spec - decimal) and generates a sample population
    of n. Returns a dataframe mostly for style'''
    
    # determine number of COVID-19 positive and negative
    cvp = round(n * p)
    cvn = round(n * (1 - p))
    
    # determine expected test results of COVID-19 +
    tp = round(sens * cvp)
    fn = round((1 - sens) * cvp)
    
    # determine expected test results of COVID-19 -
    fp = round((1 - spec) * cvn)
    tn = round(spec * cvn)
    
    # sum test results by result
    tot_p = tp + fp
    tot_n = tn + fn
    
    return pd.DataFrame(
        [[tp, fp, tot_p],
         [fn, tn, tot_n],
         [cvp, cvn, n]],
        columns=columns,
        index=index
    )

For [Indiana](https://www.reddit.com/r/science/comments/hdy7yc/a_new_study_from_penn_state_estimates_that_the/fvpil39?utm_source=share&utm_medium=web2x) on 6/23, the estimated percent of infected was 5.6% 95% CL(3.1, 8.3) from [covid-19-projections.com](https://covid19-projections.com/us-in).

In [6]:
case_generator(0.031, 0.927, 0.99)

,COVID-19,Not COVID-19,Total
Test +,287,97,384
Test -,23,9593,9616
Total,310,9690,10000


In [7]:
case_generator(0.056, 0.927, 0.99)

,COVID-19,Not COVID-19,Total
Test +,519,94,613
Test -,41,9346,9387
Total,560,9440,10000


In [8]:
case_generator(0.083, 0.927, 0.99)

,COVID-19,Not COVID-19,Total
Test +,769,92,861
Test -,61,9078,9139
Total,830,9170,10000
